In [15]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from more_itertools import chunked
from psql_methods import execute_commands
from Openseas_Methods import *
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import random
from pymongo import MongoClient
from tqdm import tqdm
from datetime import datetime
import requests
import psql_methods as psql
from IPython.display import display
import os
import pandas as pd
import alchemy_methods as alc
from image_utils import pull_image_from_url
import multiprocessing
skip_list=['ens','base-introduced','fundrop-pass','gemesis','apecoin','dai-stablecoin','uniswap','1inch-token']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
snob_df = pd.read_pickle('snob_effect_nfts.pkl')
snob_slugs = tuple(snob_df['Collection'].unique())

In [3]:
command = [f"select * from nfttoimage where slug in {snob_slugs}"]
rows  = execute_commands(command)

In [4]:
len(rows)
# Convert the list of tuples to a DataFrame
tuple_df = pd.DataFrame(rows, columns=['Collection', 'NFT_num', 'url'])
snob_df['NFT_num'] = snob_df['NFT_num'].astype(int)
# Merge the DataFrames to find tuples not already in df
merged_df = pd.merge(tuple_df, snob_df, on=['Collection', 'NFT_num'], how='left', indicator=True)

# Filter the tuples that are not in df
not_in_df = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis=1)

In [5]:
def sample_rows(group):
    if len(group) >= 300:
        return group.sample(300)
    else:
        return group
sampled_df = not_in_df.groupby('Collection', group_keys=False).apply(sample_rows)

In [6]:
grouped_data = sampled_df.groupby('Collection').apply(lambda x: (x['Collection'].iloc[0], list(zip(x['NFT_num'], x['url']))))
args = list(grouped_data)


In [ ]:
records = []
num_processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=num_processes) as pool: # Use all cores   
    for result in pool.starmap(pull_image_from_url, args):
        records.append(result)

/global/scratch/tlundy/NFT_Research/nft_venv/lib/python3.8/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/global/scratch/tlundy/NFT_Research/nft_venv/lib/python3.8/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/global/scratch/tlundy/NFT_Research/nft_venv/lib/python3.8/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/global/scratch/tlundy/NFT_Research/nft_venv/lib/python3.8/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/global/scratch/tlundy/NFT_Research/nft_venv/lib/python3.8/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be co

In [10]:
commands = ["SELECT * from collectiontoaddress"]
data = psql.execute_commands(commands)
slug_to_contract = {x[0]:x[1] for x in data}

In [12]:
sampled_df['Contract'] = sampled_df['Collection'].map(slug_to_contract)
selected_columns = sampled_df[['Contract', 'NFT_num']].to_numpy().tolist()
print(len(selected_columns))
sales_grab = list(chunked(selected_columns,10_000))

377797


In [ ]:
for sale_chunk in sales_grab:
    sales = alc.NFT_to_sales(sale_chunk)
    commands = []
    data_list = []
    command = "INSERT INTO nfttosales_2 (contract, token_id, sale_price) VALUES (%s, %s, %s) returning token_id"
    for sale in sales:
        commands.append(command)
        data_list.append(sale)
    psql.execute_commands(commands,data_list)

 16%|███████▎                                       | 1557/10000 [05:07<24:49,  5.67it/s]

In [7]:
import counterfeit_utils as cfu
df = cfu.count_overlaps('world-of-women-nft')

In [8]:
df.head(50)

,slug,sorted_order,row_count
10691,world-of-women-galaxy,1,8998
708,asian-identity-collection,2,0
10238,universeofwomen,3,1524
10704,wow-pixies-v2,4,1190
5530,lovelywomens-world,5,0
10658,women-from-venus,6,205
10685,world-of-boys-official,7,27
8954,stardustgeneration,8,71
10665,womenrise,9,1069
1803,chillennials,10,19
